# 1. 基于类构建输出格式的prompt

In [10]:
from langchain_openai import OpenAI
import pandas as pd
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain import PromptTemplate


# 定义接收的数据格式
class FlowerDescription(BaseModel):
    flower_type: str = Field(description="鲜花的种类")
    price: str = Field(description="鲜花的价格")
    description: str = Field(description="鲜花的描述文案")
    reason: str = Field(description="为什么要这样写这个文案")


output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)
output_format_prompt = output_parser.get_format_instructions()  ## 获取输出格式指示
# 打印提示
# print("输出格式：", format_instructions)
# print("============")


# todo:这个prompt有点长
'''
'\n输出格式： The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"flower_type": {"description": "鲜花的种类", "title": "Flower Type", "type": "string"}, \n"price": {"description": "鲜花的价格", "title": "Price", "type": "integer"}, \n"description": {"description": "鲜花的描述文案", "title": "Description", "type": "string"}, \n"reason": {"description": "为什么要这样写这个文案", "title": "Reason", "type": "string"}},\n "required": ["flower_type", "price", "description", "reason"]}\n```\n'
'''

'\n\'\n输出格式： The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"flower_type": {"description": "鲜花的种类", "title": "Flower Type", "type": "string"}, \n"price": {"description": "鲜花的价格", "title": "Price", "type": "integer"}, \n"description": {"description": "鲜花的描述文案", "title": "Description", "type": "string"}, \n"reason": {"description": "为什么要这样写这个文案", "title": "Reason", "type": "string"}},\n "required": ["flower_type", "price", "description", "reason"]}\n```\n\'\n'

In [13]:
# 构建完整对话
prompt_template = """您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower} ，您能提供一个吸引人的简短中文描述吗？
{output_format_prompt}"""

prompt = PromptTemplate.from_template(
    prompt_template,
    partial_variables={"output_format_prompt": output_format_prompt})  # 填充输出结构

model = OpenAI()
# 数据集结构与数据集准备
df = pd.DataFrame(columns=["flower_type", "price", "description", "reason"])
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["50", "30", "20"]

for flower, price in zip(flowers, prices):
    prompt_input = prompt.format(flower=flower, price=price)
    output = model(prompt_input)
    # 解析模型的输出：todo：output输出结构不稳定，导致解析失败报错
    parsed_output = output_parser.parse(output)
    print(parsed_output)
    df.loc[len(df)] = parsed_output.dict()  # 数据放到df中

# 打印字典
print("输出的数据：", df.to_dict(orient='records'))

AttributeError: 'str' object has no attribute 'to_string'